## Importing Libraries

In [47]:
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import re
import spacy
import torch

from nltk.tokenize.treebank import TreebankWordDetokenizer
import spacy
import nltk
import os
from skimage import io
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchviz import make_dot, make_dot_from_trace

## Reading Data & Preprocessing 

In [2]:
df = pd.read_csv("labels.csv")
df

,Unnamed: 0,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,hilarious,general,not_offensive,not_motivational,very_positive
1,1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,not_funny,general,not_offensive,motivational,very_positive
2,2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,very_funny,not_sarcastic,not_offensive,not_motivational,positive
3,3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive
4,4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral
...,...,...,...,...,...,...,...,...,...
6987,6987,image_6988.jpg,Tuesday is Mardi Gras Wednesday is Valentine's...,Tuesday is Mardi Gras Wednesday is Valentine's...,very_funny,twisted_meaning,very_offensive,motivational,neutral
6988,6988,image_6989.jpg,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,funny,twisted_meaning,not_offensive,not_motivational,neutral
6989,6989,image_6990.png,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,funny,general,slight,not_motivational,positive
6990,6990,image_6991.jpg,When I VERY have time is a fantasy No one has ...,When I have time is a fantasy. no one has time...,not_funny,twisted_meaning,not_offensive,motivational,very_positive


In [3]:
df.isna().sum()

Unnamed: 0             0
image_name             0
text_ocr             161
text_corrected         5
humour                 0
sarcasm                0
offensive              0
motivational           0
overall_sentiment      0
dtype: int64

In [4]:
df.dropna(inplace = True)
df = df.reset_index()
df

,index,Unnamed: 0,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,0,0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,hilarious,general,not_offensive,not_motivational,very_positive
1,1,1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,not_funny,general,not_offensive,motivational,very_positive
2,2,2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,very_funny,not_sarcastic,not_offensive,not_motivational,positive
3,3,3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive
4,4,4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral
...,...,...,...,...,...,...,...,...,...,...
6825,6987,6987,image_6988.jpg,Tuesday is Mardi Gras Wednesday is Valentine's...,Tuesday is Mardi Gras Wednesday is Valentine's...,very_funny,twisted_meaning,very_offensive,motivational,neutral
6826,6988,6988,image_6989.jpg,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,funny,twisted_meaning,not_offensive,not_motivational,neutral
6827,6989,6989,image_6990.png,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,funny,general,slight,not_motivational,positive
6828,6990,6990,image_6991.jpg,When I VERY have time is a fantasy No one has ...,When I have time is a fantasy. no one has time...,not_funny,twisted_meaning,not_offensive,motivational,very_positive


## Text Improvement

In [5]:
for i in range(len(df)):
    temp = df['text_corrected'][i]
    temp = re.sub(r'[^\w\s]', '', temp).lower()
    df['text_corrected'][i] = temp

C:\Users\abdul\AppData\Local\Temp/ipykernel_6408/966811553.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_corrected'][i] = temp


In [6]:
df['overall_sentiment'] = df['overall_sentiment'].replace({'very_negative':'negative', 'very_positive':'positive'})
df

,index,Unnamed: 0,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,0,0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,look there my friend lightyear now all sohalik...,hilarious,general,not_offensive,not_motivational,positive
1,1,1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,the best of 10 yearchallenge completed in less...,not_funny,general,not_offensive,motivational,positive
2,2,2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,sam thorne strippin follow follow saw everyon...,very_funny,not_sarcastic,not_offensive,not_motivational,positive
3,3,3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 year challenge sweet dee edition,very_funny,twisted_meaning,very_offensive,motivational,positive
4,4,4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 year challenge with no filter 47 hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral
...,...,...,...,...,...,...,...,...,...,...
6825,6987,6987,image_6988.jpg,Tuesday is Mardi Gras Wednesday is Valentine's...,tuesday is mardi gras wednesday is valentines ...,very_funny,twisted_meaning,very_offensive,motivational,neutral
6826,6988,6988,image_6989.jpg,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,must watch movies of 2017 iti chennai memes ma...,funny,twisted_meaning,not_offensive,not_motivational,neutral
6827,6989,6989,image_6990.png,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,less more talking planning soda junk food comp...,funny,general,slight,not_motivational,positive
6828,6990,6990,image_6991.jpg,When I VERY have time is a fantasy No one has ...,when i have time is a fantasy no one has time ...,not_funny,twisted_meaning,not_offensive,motivational,positive


In [7]:
df2 = df[['image_name','text_corrected', 'overall_sentiment', 'humour','sarcasm', 'offensive', 'motivational']]

## Label Encoding


In [8]:
encoding = LabelEncoder()
df2['overall_sentiment'] = encoding.fit_transform(df2['overall_sentiment'])
df2['humour'] = encoding.fit_transform(df2['humour'])
df2['sarcasm'] = encoding.fit_transform(df2['sarcasm'])
df2['offensive'] = encoding.fit_transform(df2['offensive'])
df2['motivational'] = encoding.fit_transform(df2['motivational'])

df2.to_csv('updated_file.csv', index = False)

C:\Users\abdul\AppData\Local\Temp/ipykernel_6408/2010538945.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['overall_sentiment'] = encoding.fit_transform(df2['overall_sentiment'])
C:\Users\abdul\AppData\Local\Temp/ipykernel_6408/2010538945.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['humour'] = encoding.fit_transform(df2['humour'])
C:\Users\abdul\AppData\Local\Temp/ipykernel_6408/2010538945.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


In [9]:
df2

,image_name,text_corrected,overall_sentiment,humour,sarcasm,offensive,motivational
0,image_1.jpg,look there my friend lightyear now all sohalik...,2,1,0,1,1
1,image_2.jpeg,the best of 10 yearchallenge completed in less...,2,2,0,1,0
2,image_3.JPG,sam thorne strippin follow follow saw everyon...,2,3,1,1,1
3,image_4.png,10 year challenge sweet dee edition,2,3,2,3,0
4,image_5.png,10 year challenge with no filter 47 hilarious ...,1,1,3,3,1
...,...,...,...,...,...,...,...
6825,image_6988.jpg,tuesday is mardi gras wednesday is valentines ...,1,3,2,3,0
6826,image_6989.jpg,must watch movies of 2017 iti chennai memes ma...,1,0,2,1,1
6827,image_6990.png,less more talking planning soda junk food comp...,2,0,0,2,1
6828,image_6991.jpg,when i have time is a fantasy no one has time ...,2,2,2,1,0


## Transforming & Padding 

In [17]:
std = [0.5, 0.4, 0.5]
mean = [0.5, 0.4, 0.5]


transform = transforms.Compose(
    [
     transforms.Resize((20, 20)),  
     transforms.ToTensor(),
     transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
    ])

In [18]:
def padding(t):
    t = torch.tensor(t)
    padding = 187 - t.size()[0]
    t = torch.nn.functional.pad(t, (0, padding))
    return t

## Text Embedding for DataLoader

In [19]:
#Inspiration for this part taken from internet
class Vocabulary:
    def __init__(self, freq_threshold):
        self.stoi = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.itos = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.freq_threshold = freq_threshold

    
    def __len__(self):
        return len(self.itos)

    @staticmethod
    def tokenizer_eng(text):
        return TreebankWordDetokenizer().detokenize(text)

    
    def build_vocabulary(self, sentence_list):
        frequencies = {}
        idx = 4

        for sentence in sentence_list:
            for word in self.tokenizer_eng(sentence):
                if word not in frequencies:
                    frequencies[word] = 1

                else:
                    frequencies[word] += 1

                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1

    
    def numericalize(self, text):
        tokenized_text = self.tokenizer_eng(text)

        return [
            float(self.stoi[token]) if token in self.stoi else float(self.stoi["<UNK>"])
            for token in tokenized_text
        ]

## Data Loader

In [21]:
class MemeDataset(Dataset):
    def __init__(self,  root_dir, csv_file, transform = None, freq_threshold = 5):
        self.df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.img = self.df['image_name']
        self.captions = self.df['text_corrected']
        
        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocabulary(list(self.captions))
    
    def __getitem__(self, index):
        
        image = Image.open(os.path.join(self.root_dir, self.df.iloc[index, 0])).convert("RGB")
        
        y_label = torch.tensor(int(self.df.iloc[index, 2]))
        y_label2 = torch.tensor(int(self.df.iloc[index, 3]))
        y_label3 = torch.tensor(int(self.df.iloc[index, 4]))
        y_label4 = torch.tensor(int(self.df.iloc[index, 5]))
        y_label5 = torch.tensor(int(self.df.iloc[index, 6]))
        
        caption = self.captions[index]
        
        if self.transform:
            image = self.transform(image)
            
        numericalized_caption = [self.vocab.stoi["<SOS>"]]
        numericalized_caption += self.vocab.numericalize(caption)
        numericalized_caption.append(self.vocab.stoi["<EOS>"])
        
        padded_text_tensor = padding(torch.tensor((numericalized_caption)))
        
        return (image, padded_text_tensor, y_label, y_label2, y_label3, y_label4, y_label5)
    
    def __len__(self):
        return len(self.df)

## Building Nueral Networks From Scratch

In [22]:
class image(nn.Module):
    def __init__(self):
        super(image, self).__init__()
        self.flat = nn.Flatten()      
        self.inn = nn.Linear(1200, 512)
        self.hidden1=nn.Linear(512, 128)
        self.hidden2=nn.Linear(128, 32)        
        self.hidden3=nn.Linear(32, 10)
        self.outt = nn.Linear(10, 3)

    def forward(self, x):
        x = self.flat(x)
        x = torch.sigmoid(self.inn(x))
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = F.relu(self.hidden3(x))
        x = self.outt(x)
      
        return x 

In [23]:
class text(nn.Module):
    def __init__(self):
        super(text, self).__init__()
        self.flat = nn.Flatten()
        self.inn = nn.Linear(187, 120)
        self.hidden1 = nn.Linear(120, 60)
        self.hidden2 = nn.Linear(60, 15)       
        self.outt = nn.Linear(15, 3)


    def forward(self, x):
        x = self.flat(x)
        x = torch.sigmoid(self.inn(x))
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = self.outt(x)
      
        return x 

In [24]:
class Combined_model(nn.Module):
    def __init__(self, modelA, modelB):
        super(Combined_model, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        
        self.classifier = nn.Linear(6, 128)
        self.classifier1 = nn.Linear(128, 64)
        self.classifier2 = nn.Linear(64, 32)
        self.classifier3 = nn.Linear(32, 16)
        
        self.out1 = nn.Linear(16, 3)
        self.out2 = nn.Linear(16, 4)
        self.out3 = nn.Linear(16, 4)
        self.out4 = nn.Linear(16, 4)
        self.out5 = nn.Linear(16, 2)
        
    def forward(self, x1, x2):
        x1 = self.modelA(x1)
        x2 = self.modelB(x2)
        temp = torch.cat((x1, x2), dim=1)
        
        temp = self.classifier(temp)
        temp = self.classifier1(temp)
        temp = self.classifier2(temp)
        temp = self.classifier3(temp)
        
        out1 = self.out1(temp)
        out2 = self.out2(temp)
        out3 = self.out3(temp)
        out4 = self.out4(temp)
        out5 = self.out5(temp)
        
        return out1, out2, out3, out4, out5

## Implementation

In [25]:
net = image()
net_text = text()
combined_model = Combined_model(net, net_text)

In [26]:
dataset = MemeDataset(root_dir = './images', csv_file = 'updated_file.csv', transform = transform)

In [27]:
train_set, test_set = torch.utils.data.random_split(dataset, [5000, 1830])
test_loader = DataLoader(dataset = test_set, batch_size = 50, shuffle = True)
train_loader = DataLoader(dataset = train_set, batch_size = 50, shuffle = True)

In [48]:
def train_loop(dataloader, model, loss_fn, optimizer):

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    acc1 = 0
    acc2 = 0
    acc3 = 0
    acc4 = 0

    for batch, (X, T, y, y2, y3, y4, y5) in enumerate(dataloader):
        
        pred1, pred2, pred3, pred4, pred5 = model(X,T)

        loss = loss_fn(pred1, y)
        loss += loss_fn(pred2, y2)
        loss += loss_fn(pred3, y3)
        loss += loss_fn(pred4, y4)
        
        acc1 += (pred1.argmax(1) == y).type(torch.float).sum().item()
        acc2 += (pred1.argmax(1) == y2).type(torch.float).sum().item()
        acc3 += (pred1.argmax(1) == y3).type(torch.float).sum().item()
        acc4 += (pred1.argmax(1) == y4).type(torch.float).sum().item()

        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"Loss in this iteration: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
    acc1 /= size
    acc2 /= size
    acc3 /= size
    acc4 /= size
    
    fs1 = f1_score(y, pred1.argmax(1), average = 'micro')
    fs2 = f1_score(y, pred2.argmax(1), average = 'micro')
    fs3 = f1_score(y, pred3.argmax(1), average = 'micro')
    fs4 = f1_score(y, pred4.argmax(1), average = 'micro')

    print(f"\n Humor Accuracy: {(100 * acc1):>0.1f}%")
    print(f"\n Humor f1 score: {(100 * fs1):>0.1f}%")
    
    
    print(f"\n Sarcasm Accuracy: {(100 * acc2):>0.1f}%")
    print(f"\n Sarcasm f1 score: {(100 * fs2):>0.1f}%")
    
    
    print(f"\n Offensive Accuracy: {(100 * acc3):>0.1f}%")
    print(f"\n Offensive f1 score: {(100 * fs3):>0.1f}%")
    
    
    print(f"\n Motivational Accuracy: {(100 * acc4):>0.1f}%")
    print(f"\n Motivational f1 score: {(100 * fs4):>0.1f}%")

In [49]:
optim = torch.optim.Adam(combined_model.parameters(), lr = 0.03)

loss = nn.CrossEntropyLoss()

epochs = 1

for t in range(epochs):
    
    print(f"\nEpoch {t+1}:\n")
    
    train_loop(train_loader, combined_model, loss, optim)


Epoch 1:



C:\Users\abdul\AppData\Local\Temp/ipykernel_6408/593810071.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t = torch.tensor(t)


Loss in this iteration: 4.758005  [    0/ 5000]
Loss in this iteration: 4.577103  [  500/ 5000]
Loss in this iteration: 4.841576  [ 1000/ 5000]
Loss in this iteration: 4.368140  [ 1500/ 5000]
Loss in this iteration: 4.403984  [ 2000/ 5000]
Loss in this iteration: 4.727898  [ 2500/ 5000]
Loss in this iteration: 4.845053  [ 3000/ 5000]
Loss in this iteration: 4.715659  [ 3500/ 5000]
Loss in this iteration: 4.508129  [ 4000/ 5000]
Loss in this iteration: 4.453217  [ 4500/ 5000]

 Humor Accuracy: 58.2%

 Humor f1 score: 60.0%

 Sarcasm Accuracy: 23.7%

 Sarcasm f1 score: 8.0%

 Offensive Accuracy: 21.9%

 Offensive f1 score: 8.0%

 Motivational Accuracy: 37.3%

 Motivational f1 score: 60.0%


## Fin 